In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'cpu-cluster'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.5, 1, 5, 10, 20),
    '--max_iter': choice(50, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training',
              entry_script='train.py',
              compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs = 4  
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print('Best run id :', best_run.id)
print('Accuracy ', best_run_metric['Accuracy'])

model = best_run.register_model(model_name='model', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
import pandas as pd

data = pd.concat([x, y], axis=1)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_d, test_d = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
train_d.to_csv('training/train_data.csv')

In [ ]:
automl_exp = Experiment(ws, name='automl_exp')

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='training/', target_path='data/')

In [ ]:
from azureml.core import Dataset
train_data = Dataset.Tabular.from_delimited_files(path=[(datastore,('data/train_data.csv'))])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target)

In [2]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_automl_run, best_model = automl_run.get_output()
best_automl_run.register_model(model_name = "automl.pkl", model_path = './outputs/')
print(best_model._final_estimator)